In [0]:
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools   

In [2]:
fileId = '1-XpPnKEH-DVYeRr69l_B7Jflh_SthK_S'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

Extracted zip file 1-XpPnKEH-DVYeRr69l_B7Jflh_SthK_S.zip


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
import matplotlib.pyplot as plt

In [7]:
import os
import gc
#import cv2
import json
import time

import numpy as np
import pandas as pd
from pathlib import Path
from keras.utils import to_categorical

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
!ls

adc.json	sample_data		   Train_data
cat_levels.csv	TIL_sample_submission.csv  Train_data.csv


In [0]:
import dask.dataframe as pd2
loc0 = (r'/content/Train_data/Train_data.csv')
#train = pd2.read_csv(loc0,error_bad_lines=False,sep='|',warn_bad_lines=False,assume_missing=True,chunksize=300)

In [0]:
#loc0 = (r'/content/Train_data.csv')
loc1 = (r'/content/cat_levels.csv')
#train = pd.read_csv(loc0, error_bad_lines=False)
levels = pd.read_csv(loc1, error_bad_lines=False)
loc2 = (r'/content/TIL_sample_submission.csv')
sample = pd.read_csv(loc2, error_bad_lines=False)

In [11]:
sample

,id,category_tree
0,21047,entertainment->bollywood
1,21047,News & Politics -> Politics -> bjp -> dharmend...
2,243958,sports -> Outdoor Games - > Tennis -> novak dj...
3,243958,location -> serbia


In [57]:
levels[:]

,level1_categories,level_2_categories
0,Info Tech,Info Tech
1,Travel & Tourism,PaaS
2,Travel,Products
3,Vehicles & Parts,SaaS
4,Automobiles,Travel & Tourism
...,...,...
157,NaN,CAN
158,NaN,Click
159,NaN,DFP
160,NaN,Impression


In [3]:
pip install googletrans

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=51b819825f85f00c0ca21806ba6646891d6d5f2fb30cb9c651d51acecd646662
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [4]:
from googletrans import Translator
import nltk
nltk.download('all')
from nltk.corpus import wordnet

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [73]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

list4=[]
dd=-1
id_=[]
tree = []
for chunk in pd.read_csv(loc0,sep='|', chunksize=1):
  if(dd ==2000):
    break
  dd=dd+1
  if(dd%10==0):
    print(dd)
  if(chunk.shape[1]!=5):
    continue
  chunk.iloc[0][1] = chunk.iloc[0][1].replace("/", " ")
  chunk.iloc[0][1] = chunk.iloc[0][1].replace(".", " ")
  chunk.iloc[0][1] = chunk.iloc[0][1].replace("-", " ")
  sw = stopwords.words('english')
  list3=[]
  list2=[]
  for j in range (0,32):     
    Y_list = word_tokenize(levels['level1_categories'][j].lower())
    Y_set = {w for w in Y_list if not w in sw} 

    z=0

    for jj in range(0,4):
#      if(len(chunk.iloc[0][jj])==0):
#        continue
      X_list = word_tokenize(str(chunk.iloc[0][jj]).lower()) 
      X_set = {w for w in X_list if not w in sw}
    
      q=0
#    z=0    
#      list1=[]
      for word1 in X_set:
       for word2 in Y_set:
         if(len(wordnet.synsets(word1))):
           wordFromList1 = wordnet.synsets(word1)[0]
         else:
           continue
         if(len(wordnet.synsets(word2))):
           wordFromList2 = wordnet.synsets(word2)[0]
         else:
           continue
         q =  wordFromList1.wup_similarity(wordFromList2)
         if(q!=None):
          z=z+q
  #        list1.append(q)
    list2.append(z)
#    if(z>45):
#      if(levels['level1_categories'][j] == 'Info Tech' and z>32):
#        list3.append(levels['level1_categories'][j])
#      elif(levels['level1_categories'][j] != 'Info Tech'):
#        list3.append(levels['level1_categories'][j])
  if(len(list3)==0):
    max_v = 0
    max_i = 0
    for uu in range(0,32):
      if(list2[uu]>max_v):
        max_v=list2[uu]
        max_i=uu
    list3.append(levels['level1_categories'][max_i])


  for j in range (0,len(levels)):     
    Y_list = word_tokenize(levels['level_2_categories'][j].lower())
    Y_set = {w for w in Y_list if not w in sw} 

    z=0

    for jj in range(0,4):
#      if(len(chunk.iloc[0][jj])==0):
#        continue
      X_list = word_tokenize(str(chunk.iloc[0][jj]).lower()) 
      X_set = {w for w in X_list if not w in sw}
    
      q=0
#    z=0    
#      list1=[]
      for word1 in X_set:
       for word2 in Y_set:
         if(len(wordnet.synsets(word1))):
           wordFromList1 = wordnet.synsets(word1)[0]
         else:
           continue
         if(len(wordnet.synsets(word2))):
           wordFromList2 = wordnet.synsets(word2)[0]
         else:
           continue
         q =  wordFromList1.wup_similarity(wordFromList2)
         if(q!=None):
          z=z+q
  #        list1.append(q)
    list2.append(z)
  if(len(list3)==1):
    max_v = 0
    max_i = 0
    for uu in range(0,len(levels)):
      if(list2[uu]>max_v):
        max_v=list2[uu]
        max_i=uu
 #   list3.append(levels['level_2_categories'][max_i])
    list3[0] = list3[0] + ' -> ' + levels['level_2_categories'][max_i]


  list4.append(list3)
  id_.append(chunk.iloc[0][4])
#print(q)
#print(z)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000


In [51]:
#list2

[8.913842890855276,
 6.517153177060298,
 3.2585765885301488,
 8.123231996761408,
 3.286611135682343,
 3.0646304675716447,
 6.517153177060298,
 0,
 7.211132984662394,
 3.479070929070929,
 6.467390942390944]

In [64]:
list4[:10]

[['Music and Audio -> News'],
 ['Vehicles & Parts -> Education'],
 ['Vehicles & Parts -> Education'],
 ['Career & Job -> Religion & Spirituality'],
 ['Family and Relationships -> Religion & Spirituality'],
 ['Career & Job -> Religion & Spirituality'],
 ['Real Estate -> Religion & Spirituality'],
 ['Music and Audio -> News'],
 ['Music and Audio -> Religion & Spirituality'],
 ['Music and Audio -> Religion & Spirituality']]

In [55]:
id_[:5]

[2674,
 3874,
 425,
 5177,
 3260,
 1867,
 4988,
 2882,
 2675,
 2058,
 2676,
 3875,
 0,
 2481,
 1262,
 4615,
 4616,
 4229,
 629,
 3261,
 3677,
 6523,
 3476]

In [0]:
sub = pd.DataFrame()
sub['id'] = id_
sub['category_tree'] = list4
sub['category_tree'] = sub['category_tree'].str[0]

In [85]:
sub.head(10)

,id,category_tree
0,2674,Music and Audio -> News
1,3874,Vehicles & Parts -> Education
2,425,Vehicles & Parts -> Education
3,5177,Career & Job -> Religion & Spirituality
4,3260,Family and Relationships -> Religion & Spiritu...
5,1867,Career & Job -> Religion & Spirituality
6,4988,Real Estate -> Religion & Spirituality
7,2882,Music and Audio -> News
8,2675,Music and Audio -> Religion & Spirituality
9,2058,Music and Audio -> Religion & Spirituality


In [81]:
len(sub)

2001

In [0]:
sub.to_csv('submission.csv',index=False)

In [0]:
# This block is only relevant if you are running this notebook on Google Colab
from google.colab import files
files.download('submission.csv')